In [1]:
import os
import sqlite3
import sketch
import openai
import plotly.express as px # Needed to import this as px so an error was not thrown in the generated code
import kaleido
import pandas as pd
from pandas_llm import PandasLLM
from pandasai import SmartDataframe
from pandasai.llm import OpenAI
from collections import namedtuple
from pathlib import Path, PosixPath
from typing import NamedTuple

In [6]:
# NamedTuple type hint
class ParametersType(NamedTuple):
    data_dir: PosixPath # Platform neutral pathlib PosixPath to data directory
    acs_path: PosixPath # Platform neutral pathlib PosixPath to ACS data
    db_path: PosixPath # Platform neutral pathlib PosixPath to SQLite3 database
    db_connection: sqlite3.Connection # SQLite3 database connection
    openai_api_key: str # OpenAI API key

In [7]:
Parameters: ParametersType = ParametersType(
    data_dir = Path.cwd() / "Data",
    acs_path = Path.cwd() / "Data/ACS_2012_21.csv",
    db_path= Path.cwd() / "Data/data.sqlite3",
    db_connection = sqlite3.connect(Path.cwd() / "Data/data.sqlite3"),  # ":memory:"
    openai_api_key = os.environ["OPENAI_API_KEY"],
)

# List all tables in SQLite database

In [4]:
cursor = Parameters.db_connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables: list[tuple] = cursor.fetchall()
tables

[('saacempratio',),
 ('saacartsva',),
 ('saacemplq',),
 ('saacvaratio',),
 ('saaccompratio',),
 ('saacartsemp',),
 ('saacvalq',),
 ('saacartscomp',),
 ('saaccomplq',),
 ('sasummary',),
 ('saocomp',),
 ('saova',),
 ('saoactva',),
 ('saoemp',),
 ('sapce4',),
 ('sapce3',),
 ('sapce1',),
 ('sapce2',),
 ('sainc7h',),
 ('saemp25s',),
 ('sainc35',),
 ('sainc51',),
 ('sainc50',),
 ('saemp25n',),
 ('sainc7n',),
 ('sainc7s',),
 ('sainc1',),
 ('saemp27n',),
 ('sainc6n',),
 ('sainc5n',),
 ('sainc5h',),
 ('sainc40',),
 ('saemp27s',),
 ('sainc5s',),
 ('sainc70',),
 ('sainc91',),
 ('sainc6s',),
 ('sainc4',),
 ('sainc30',),
 ('sainc45',),
 ('sagdp4n',),
 ('sagdp11n',),
 ('sagdp2n',),
 ('sagdp6n',),
 ('sagdp9n',),
 ('sagdp1',),
 ('sagdp3n',),
 ('sagdp8n',),
 ('sagdp7n',),
 ('sagdp5n',),
 ('acs',),
 ('saacempratio_definitions',),
 ('saacartscomp_definitions',),
 ('saacemplq_definitions',),
 ('saacvalq_definitions',),
 ('saacvaratio_definitions',),
 ('saaccompratio_definitions',),
 ('saacartsva_definitio

# ACS Data

In [ ]:
acs_df: pd.DataFrame = pd.read_csv(Parameters.acs_path)
acs_df.drop(columns=["Unnamed: 0"], inplace=True)
display(acs_df.info())
display(acs_df.head())

# Test OpenAI API Key

In [ ]:
"""
Please provide python code to ask openai a question and retrieve the answer

To ask OpenAI a question and retrieve the answer, you can use the OpenAI API.
Here's an example Python code that demonstrates how to do this using the openai package:

In this example, you need to replace "YOUR_API_KEY" with your actual API key,
which you can obtain by signing up for OpenAI's API at https://beta.openai.com/signup/.

The ask_openai function takes three parameters: the question you want to ask,
the name of the OpenAI model you want to use (e.g., "davinci" or "curie"),
and the max_length of the generated answer (in number of tokens).

The function sends a request to the OpenAI API using the openai.Completion.create method,
which takes the engine, prompt, and max_tokens as parameters. The response is a list of
completions (i.e., possible answers), and we take the first one (which is usually the most likely answer)
and return it as a string.
"""

openai.api_key = Parameters.openai_api_key # "YOUR_API_KEY" # replace with your API key

def ask_openai(question, model, max_length):
    prompt = f"Q: {question}\nA:"
    completions = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=max_length
    )
    answer = completions.choices[0].text.strip()
    return answer

# Example usage
question = "What is the capital of France?"
model = "davinci"
max_length = 100
answer = ask_openai(question, model, max_length)
print(answer)

# [LangChain](https://pypi.org/project/langchain/)
https://coinsbench.com/chat-with-your-databases-using-langchain-bb7d31ed2e76  
https://medium.com/@hannanmentor/python-custom-chatgpt-with-your-own-data-f307635dd5bd  

In [ ]:
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain

## Check that LangChain works

In [ ]:
# Simple LLM call Using LangChain
llm = OpenAI(model_name="text-davinci-003", openai_api_key=Parameters.openai_api_key)
question = "Which language is used to create chatgpt ?"
print(question, llm(question))

## Prompt template

In [ ]:
# Creating a prompt template and running the LLM chain
template = "What are the top {n} resources to learn {language} programming?"
prompt = PromptTemplate(template=template,input_variables=['n','language'])
chain = LLMChain(llm=llm,prompt=prompt)
input = {'n':3,'language':'Python'}
print(chain.run(input))

In [ ]:
from langchain import OpenAI, SQLDatabase  #, SQLDatabaseChain, PromptTemplate, LLMChain

In [ ]:
# SQLDatabase.create_table_from_df(df=acs_df, table_name="acs", db_connection=Parameters.db_connection)  # Create table from dataframe

In [ ]:
# Use LangChain to answers quetions using a SQLite3 database
llm = OpenAI(model_name="text-davinci-003", openai_api_key=Parameters.openai_api_key)
db = SQLDatabase(db_connection=Parameters.db_connection)
question = "What is the population of Kentucky?"
print(question, llm(question,db))

In [24]:
from langchain.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [25]:
from langchain.utilities import SQLDatabase

In [27]:
db = SQLDatabase.from_uri("sqlite:///Data/data.sqlite3")

In [30]:
def get_schema(_):
    return db.get_table_info()

In [31]:
def run_query(query):
    return db.run(query)

In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable.passthrough import RunnablePassthrough  # Try without .passthrough

In [38]:
model = ChatOpenAI()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

AttributeError: type object 'RunnablePassthrough' has no attribute 'assign'

In [ ]:

# setup llm
llm = OpenAI(temperature=0, openai_api_key=Parameters.openai_api_key)

# Create db chain
QUERY = """
Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

{question}
"""

# Setup the database chain
db_chain = SQLDatabaseChain(llm=llm, database=Parameters.db_connection, verbose=True)


def get_prompt():
    print("Type 'exit' to quit")

    while True:
        prompt = input("Enter a prompt: ")

        if prompt.lower() == 'exit':
            print('Exiting...')
            break
        else:
            try:
                question = QUERY.format(question=prompt)
                print(db_chain.run(question))
            except Exception as e:
                print(e)

get_prompt()